# Trabajo Práctico Integrador - Analisis de Datos

## Introducción
En este trabajo práctico, aplicamos las técnicas vistas en clase con un objetivo claro: explorar, limpiar y preparar un conjunto de datos reales. El fin último es sentar las bases para plantear un problema de aprendizaje supervisado.

### Nuestro Equipo:
- Maximiliano Christener
- Luiz Diaz Charris
- Mariano Fagre
- Juan Pablo Skobalski
- Ronald Uthurralt

### El Dataset:
Para este análisis, seleccionamos el UCDP Georeferenced Event Dataset (GED) Global versión 25.1, un set de datos provisto por el Uppsala Conflict Data Program (UCDP).

Elegimos este conjunto de datos por su gran detalle. Contiene información georreferenciada sobre eventos de violencia organizada que han ocurrido en todo el mundo desde 1989. Cada registro no es solo un número, sino que representa un evento violento individual, describiendo su fecha, ubicación precisa (latitud y longitud), el tipo de violencia (conflicto estatal, no estatal o unilateral), quiénes fueron los actores involucrados y una estimación de las víctimas fatales, entre otros atributos.

Lo valioso de estos datos es que nos permiten explorar fenómenos muy complejos, como los conflictos armados y la violencia política, desde dos ángulos clave: temporalmente (cómo ha evolucionado la violencia) y espacialmente (dónde se concentran).

## Nuestra Hipótesis y Objetivos
Tras una primera exploración de los datos, formulamos la siguiente hipótesis de trabajo:

### DISUCTAMOS SI CORRESPONDE O NO, SOLO LO ESCRIBO CON EL OBJETIVO DE ESTRUCTURAR EL DOCUMENTO
Hipótesis: Creemos que ciertos factores estructurales, como el tipo de violencia, la región geográfica y los actores involucrados, influyen de manera significativa en el nivel de letalidad (es decir, la cantidad de víctimas fatales) de un conflicto.

Basados en esta idea, nuestros objetivos para este trabajo son:

1. Analizar las características generales del dataset, entendiendo su distribución tanto geográfica como temporal.

2. Identificar si existen patrones de violencia claros según el tipo de evento, la región o los actores.

3. Preparar el conjunto de datos para un posible modelo de aprendizaje supervisado. Aquí, nuestra variable objetivo será best_est (la cantidad estimada de víctimas), con la que buscaremos predecir la letalidad de un evento basándonos en sus características.



## Exploración y comprensión de los datos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Esta libreria son para el heat map, me parecio copada, pero puede haber otras
import folium
from folium.plugins import HeatMap

In [ ]:
df = pd.read_csv("data/GEDEvent_v25_1.csv", low_memory=False)

df.head()

In [ ]:
print(f"Número de observaciones: {df.shape[0]}")
print(f"Número de variables: {df.shape[1]}")

df.info()

## Descripción de las columnas del dataset UCDP GED v25.1 (LAS DESCRIPCIONES LAS HICE CON EL CHATO, VEAMOSLAS JUNTOS EN DETALLE)

| **Columna** | **Descripción** |
|--------------|----------------|
| **id** | Identificador único del evento. |
| **relid** | ID del evento dentro de su conflicto (relación interna). |
| **year** | Año en que ocurrió el evento. |
| **active_year** | Indica si el conflicto estuvo activo ese año (`True/False`). |
| **code_status** | Estado del registro (ej. “Clear” o “Unclear”). |
| **type_of_violence** | Tipo de violencia (1 = interestatal, 2 = no estatal, 3 = unidireccional). |
| **conflict_dset_id** | ID interno del conflicto dentro del dataset. |
| **conflict_new_id** | Versión nueva del ID del conflicto. |
| **conflict_name** | Nombre del conflicto (ej. “Afghanistan: Government”). |
| **dyad_dset_id** | ID de la díada (par de actores enfrentados). |
| **dyad_new_id** | ID actualizado de la díada. |
| **dyad_name** | Nombres de los actores enfrentados. |
| **side_a_dset_id** | ID interno del actor A. |
| **side_a_new_id** | ID nuevo del actor A. |
| **side_a** | Nombre del actor A (ej. “Government of Iraq”). |
| **side_b_dset_id** | ID interno del actor B. |
| **side_b_new_id** | ID nuevo del actor B. |
| **side_b** | Nombre del actor B (ej. “IS”). |
| **number_of_sources** | Número de fuentes utilizadas para registrar el evento. |
| **source_article** | Texto con los artículos fuente. |
| **source_office** | Fuente institucional (ej. Reuters, AFP). |
| **source_date** | Fecha de las fuentes. |
| **source_headline** | Título de las noticias fuente. |
| **source_original** | Fuente original o primaria. |
| **where_prec** | Precisión del lugar (1 = exacta, >1 = aproximada). |
| **where_coordinates** | Coordenadas como texto. |
| **where_description** | Descripción textual del lugar. |
| **adm_1** | División administrativa 1 (provincia, estado). |
| **adm_2** | División administrativa 2 (distrito, municipio). |
| **latitude** | Latitud del evento. |
| **longitude** | Longitud del evento. |
| **geom_wkt** | Geometría en formato WKT (texto). |
| **priogrid_gid** | ID de celda de grilla espacial PRIO. |
| **country** | País donde ocurrió el evento. |
| **country_id** | Código numérico del país. |
| **region** | Región geográfica (ej. Asia, África). |
| **event_clarity** | Nivel de claridad o precisión del evento (1 = claro, >1 = incierto). |
| **date_prec** | Precisión temporal (1 = exacta, valores mayores = más incierto). |
| **date_start** | Fecha de inicio del evento. |
| **date_end** | Fecha de fin del evento. |
| **deaths_a** | Muertes del actor A. |
| **deaths_b** | Muertes del actor B. |
| **deaths_civilians** | Muertes de civiles. |
| **deaths_unknown** | Muertes sin categoría asignada. |
| **best** | Estimación principal de muertes totales. |
| **high** | Estimación alta de muertes totales. |
| **low** | Estimación baja de muertes totales. |
| **gwnoa** | Código numérico del país del actor A. |
| **gwnob** | Código numérico del país del actor B. |


# EN MI OPINION ESTAS COLUMNAS VOLEMOSLAS (no les veo utilidad):

[
 'relid', 'conflict_dset_id', 'conflict_new_id',
 'dyad_dset_id', 'dyad_new_id',
 'side_a_dset_id', 'side_a_new_id',
 'side_b_dset_id', 'side_b_new_id',
 'source_article', 'source_date', 'source_headline', 'source_original',
 'where_coordinates', 'geom_wkt', 'priogrid_gid',
 'country_id', 'gwnoa', 'gwnob'
]

# Esto para la parte de limpieza

### Tipos de Violencia

En la pagina de donde se obtuvieron los datos se expresa que los tipos de violencia se correlacionan de esta manera con los valores 1, 2 y 3:
| **Código** | **Tipo de violencia**     | **Descripción** |
|:-----------:|:--------------------------|:----------------|
| **1** | State-based conflict | Conflictos en los que al menos una de las partes es el gobierno de un Estado (por ejemplo, guerras civiles o interestatales). |
| **2** | Non-state conflict | Enfrentamientos armados entre grupos organizados no estatales, sin participación directa de un gobierno (por ejemplo, milicias rivales o grupos étnicos armados). |
| **3** | One-sided violence | Violencia dirigida contra civiles, perpetrada por un gobierno o por un grupo armado organizado (por ejemplo, masacres, represión violenta). |

In [ ]:
df.describe()

df["type_of_violence"].value_counts()

# creamos el mapping
violence_mapping = {
    1: "State-based conflict",
    2: "Non-state conflict",
    3: "One-sided violence"
}

df["type_of_violence_desc"] = df["type_of_violence"].map(violence_mapping)

In [ ]:
missing = df.isnull().mean().sort_values(ascending=False)
missing[missing > 0].head(20)

In [ ]:
# Con este grafico podemos detectar outliers
plt.figure(figsize=(6,4))
df.boxplot(column="best")
plt.title("Distribución de muertes estimadas por evento (best)")
plt.show()

## Aplicación de técnicas de visualización

### Distribución temporal de los eventos

In [ ]:
sns.set_theme(style="whitegrid", palette="muted")

plt.figure(figsize=(12,5))
sns.countplot(x="year", data=df, color="steelblue")
plt.xticks(rotation=90)
plt.title("Distribución de eventos de violencia por año")
plt.xlabel("Año")
plt.ylabel("Cantidad de eventos")
plt.show()

### Distribución por tipo de violencia

In [ ]:
plt.figure(figsize=(7,5))
sns.countplot(x="type_of_violence_desc", data=df, order=["State-based conflict", "Non-state conflict", "One-sided violence"])
plt.title("Distribución de eventos según tipo de violencia")
plt.xlabel("Tipo de violencia")
plt.ylabel("Cantidad de eventos")
plt.xticks(rotation=25)
plt.show()

### Regiones con mayor número de eventos

In [ ]:
plt.figure(figsize=(10,5))
top_regions = df["region"].value_counts().head(10)
sns.barplot(x=top_regions.index, y=top_regions.values, color="darkcyan")
plt.title("Top 10 regiones con más eventos de violencia")
plt.xlabel("Región")
plt.ylabel("Cantidad de eventos")
plt.xticks(rotation=45)
plt.show()

### Distribución geográfica

In [ ]:
m = folium.Map(location=[20, 0], zoom_start=2)

coords = df[['latitude', 'longitude', 'best']].dropna()

HeatMap(coords.values.tolist(), radius=5, blur=4).add_to(m)

m

### Distribución de víctimas fatales

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df["best"], bins=50, kde=True, color="salmon")
plt.xlim(0, 200)
plt.title("Distribución de muertes estimadas por evento")
plt.xlabel("Muertes estimadas (best)")
plt.ylabel("Frecuencia")
plt.show()

### Correlación entre variables numéricas

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df[["deaths_a","deaths_b","deaths_civilians","best","high","low"]].corr(), annot=True, cmap="YlOrBr")
plt.title("Matriz de correlación entre variables de muertes")
plt.show()
